## Dependencies

In [1]:
from databrew import DataBrew
from glue import GlueWrapper
import boto3
import json
import time
import sys
import numpy

sys.path.append("/home/glue_user/project_lf/ETL-TDD")

glue = GlueWrapper('us-east-1')
databrew = DataBrew('us-east-1')

# Run 1
import stage_claim_into_raw
import stage_policyholder_into_raw
import stage_provider_into_raw

# Run 2
import stage_claim_into_access
import stage_policyholder_into_access
import stage_provider_into_access

# Run 3
import stage_location_into_optimised
import stage_procedure_into_optimised

# Run 4
import stage_policyholder_into_optimised
import stage_provider_into_optimised


# Run 5
import stage_claim_into_optimised


# import glue dependencies
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, DataFrame
from etl.paths.components import Bucket




## Initialise Spark

In [2]:
# initialise spark session
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Reload4jLoggerFactory]
log4j:WARN No appenders could be found for logger (org.apache.hadoop.metrics2.lib.MutableMetricsFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.
/home/glue_us

## Delete objects in S3

In [2]:
# delete everything in the raw, access and optimised tiers
import boto3
s3 = boto3.resource('s3')
bucket = s3.Bucket('test-lf-wm')

# ********************* RESET RAW *********************
for obj in bucket.objects.filter(Prefix='etl/raw'):
    # s3.Object(bucket.name,obj.key).delete()

# **********************RESET ACCESS ******************
for obj in bucket.objects.filter(Prefix='etl/access'):
    # s3.Object(bucket.name,obj.key).delete()

# ******************** RESET OPTIMISED ****************
for obj in bucket.objects.filter(Prefix='etl/optimised'):
    # s3.Object(bucket.name,obj.key).delete()
    


## Run 1

In [13]:
outputs = []

raw_df, path = stage_claim_into_raw.run(spark, Bucket.TEST)
outputs.append( (raw_df, path) )
raw_df, path = stage_policyholder_into_raw.run(spark, Bucket.TEST)
outputs.append( (raw_df, path) )
raw_df, path = stage_provider_into_raw.run(spark, Bucket.TEST)
outputs.append( (raw_df, path) )

getInstance(**kwargs={}) called
endpoint is s3(https://s3.amazonaws.com)


getInstance(**kwargs={}) called
endpoint is s3(https://s3.amazonaws.com)


getInstance(**kwargs={}) called
endpoint is s3(https://s3.amazonaws.com)


In [17]:

# flatten the outputs with numpy
dfs = numpy.array(outputs).flatten()[::2]
paths = numpy.array(outputs).flatten()[1::2]

In [18]:
for df in dfs:
    df.printSchema()

root
 |-- claim_id: integer (nullable = true)
 |-- policy_holder_id: integer (nullable = true)
 |-- provider_id: integer (nullable = true)
 |-- date_of_service: string (nullable = true)
 |-- procedure: string (nullable = true)
 |-- total_procedure_cost: float (nullable = true)
 |-- medibank_pays: float (nullable = true)
 |-- medicare_pays: float (nullable = true)
 |-- excess: float (nullable = true)
 |-- out_of_pocket: float (nullable = true)

root
 |-- policy_holder_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- address: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- email_address: string (nullable = true)
 |-- insurance_plan_details: string (nullable = true)
 |-- coverage_start_date: string (nullable = true)
 |-- coverage_end_date: string (nullable = true)
 |-- policy_standing: string (nullable = true)

ro

In [7]:
example_path = paths[-1]

# Crawl
glue.create_crawler(example_path)

# glue.start_crawler(example_path)

############## Wait for crawler to finish #####################

glue.wait_for_crawler(example_path)

############## After crawlers finish. #####################

for path in paths:
     
    databrew.create_dataset(path)
    databrew.create_profile_job(path)
    databrew.start_job_run(path)

############## Wait for job to finish #####################

databrew.wait_for_job(example_path)

for path in paths:
    print(f"You can view the data profile for {path} here:\n\t{databrew.get_profile_link(path)}")
    # databrew.show_data_profile_link(path)


Couldn't create crawler. 618572314333:test-raw already exists


Crawler not running.


Couldn't create dataset. Dataset test-raw-claim already exists.
Couldn't create data profile job. The job test-raw-claim already exists.
Couldn't create dataset. Dataset test-raw-provider already exists.
Couldn't create data profile job. The job test-raw-provider already exists.


Waiting for job to finish.........................
You can view the data profile for s3://test-lf-wm/etl/raw/claim_db/claim/full/202306051005/ here:
	https://us-east-1.console.aws.amazon.com/databrew/home?region=us-east-1#dataset-details?dataset=test-raw-claim&tab=profile-overview
You can view the data profile for s3://test-lf-wm/etl/raw/claim_db/policyholder/full/202306051005/ here:
	https://us-east-1.console.aws.amazon.com/databrew/home?region=us-east-1#dataset-details?dataset=test-raw-policyholder&tab=profile-overview
You can view the data profile for s3://test-lf-wm/etl/raw/claim_db/provider/full/202306051006/ here:
	https://us-east-1.console.aws.amazon.com/databrew/home?region=us-east-1#dataset-details?dataset=test-raw-provider&tab=profile-overview


## Run 2

In [12]:
outputs = []

access_df, path = stage_claim_into_access.run(spark, Bucket.TEST)
outputs.append( (access_df, path) )
access_df, path = stage_policyholder_into_access.run(spark, Bucket.TEST)
outputs.append( (access_df, path) )
access_df, path = stage_provider_into_access.run(spark, Bucket.TEST)
outputs.append( (access_df, path) )

getInstance(**kwargs={}) called
endpoint is s3(https://s3.amazonaws.com)
s3.Bucket(name='test-lf-wm') etl/raw/claim_db/claim/full


getInstance(**kwargs={}) called
endpoint is s3(https://s3.amazonaws.com)
s3.Bucket(name='test-lf-wm') etl/raw/claim_db/policyholder/full


getInstance(**kwargs={}) called
endpoint is s3(https://s3.amazonaws.com)
s3.Bucket(name='test-lf-wm') etl/raw/claim_db/provider/full


In [19]:

# flatten the outputs with numpy
access_dfs = numpy.array(outputs).flatten()[::2]
access_paths = numpy.array(outputs).flatten()[1::2]

In [21]:
for df in access_dfs:
    df.printSchema()

root
 |-- claim_id: integer (nullable = true)
 |-- policy_holder_id: integer (nullable = true)
 |-- provider_id: integer (nullable = true)
 |-- date_of_service: string (nullable = true)
 |-- procedure: string (nullable = true)
 |-- total_procedure_cost: float (nullable = true)
 |-- medibank_pays: float (nullable = true)
 |-- medicare_pays: float (nullable = true)
 |-- excess: float (nullable = true)
 |-- out_of_pocket: float (nullable = true)

root
 |-- policy_holder_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- address: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- email_address: string (nullable = true)
 |-- insurance_plan_details: string (nullable = true)
 |-- coverage_start_date: string (nullable = true)
 |-- coverage_end_date: string (nullable = true)
 |-- policy_standing: string (nullable = true)

ro

In [15]:
example_path = access_paths[-1]

# Crawl
glue.create_crawler(example_path)
glue.start_crawler(example_path)
glue.wait_for_crawler(example_path)

############## After crawlers finish. #####################
for path in access_paths:
     
    response = databrew.create_dataset(path)
    response = databrew.create_profile_job(path)        
    response = databrew.start_job_run(path)

############## Wait for job to finish #####################
databrew.wait_for_job(example_path)

# Print links.
for path in paths:
    print(f"You can view the data profile for {path} here:\n\t{databrew.get_profile_link(path)}")
    # databrew.show_data_profile_link(path)




Waiting for crawler to finish...........
Waiting for job to finish...........................
You can view the data profile for s3://test-lf-wm/etl/access/claim_db/claim/full/202306051354/ here:
	https://us-east-1.console.aws.amazon.com/databrew/home?region=us-east-1#dataset-details?dataset=test-access-claim&tab=profile-overview
You can view the data profile for s3://test-lf-wm/etl/access/claim_db/policyholder/full/202306051355/ here:
	https://us-east-1.console.aws.amazon.com/databrew/home?region=us-east-1#dataset-details?dataset=test-access-policyholder&tab=profile-overview
You can view the data profile for s3://test-lf-wm/etl/access/claim_db/provider/full/202306051355/ here:
	https://us-east-1.console.aws.amazon.com/databrew/home?region=us-east-1#dataset-details?dataset=test-access-provider&tab=profile-overview


In [ ]:
############## Use data profile. #####################

databrew.get_dq_results(paths[0])

## Run 3

In [11]:
run3 = []
df, path = stage_location_into_optimised.run(spark,Bucket.TEST)
run3.append( (df, path) )
df, path = stage_procedure_into_optimised.run(spark,Bucket.TEST)
run3.append( (df, path) )

getInstance(**kwargs={}) called
kwargs is {'region_name': 'us-east-1'}
endpoint is s3(https://s3.amazonaws.com)
s3.Bucket(name='test-lf-wm') etl/access/claim_db/policyholder/full


getInstance(**kwargs={}) called
endpoint is s3(https://s3.amazonaws.com)
s3.Bucket(name='test-lf-wm') etl/access/claim_db/provider/full


getInstance(**kwargs={}) called
endpoint is s3(https://s3.amazonaws.com)
s3.Bucket(name='test-lf-wm') etl/access/claim_db/claim/full


In [22]:
run3_dfs = numpy.array(run3).flatten()[0::2]
run3_paths = numpy.array(run3).flatten()[1::2]

run3_paths, run3_dfs

(array(['s3://test-lf-wm/etl/optimised/location_dim/full/202306051807/',
        's3://test-lf-wm/etl/optimised/procedure_dim/full/202306051808/'],
       dtype=object),
 array([DataFrame[location_key: bigint, address: string, street: string, postcode: string, suburb: string, track_hash: bigint, record_start_ts: timestamp, record_end_ts: timestamp, record_active_flag: smallint, record_upd_ts: timestamp, record_insert_ts: timestamp],
        DataFrame[procedure: string, procedure_key: bigint, track_hash: bigint, record_start_ts: timestamp, record_end_ts: timestamp, record_active_flag: smallint, record_upd_ts: timestamp, record_insert_ts: timestamp]],
       dtype=object))

In [25]:
for run in run3_dfs:
    run.printSchema()
    run.show(5, truncate=False)

root
 |-- location_key: long (nullable = false)
 |-- address: string (nullable = true)
 |-- street: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- suburb: string (nullable = true)
 |-- track_hash: long (nullable = true)
 |-- record_start_ts: timestamp (nullable = true)
 |-- record_end_ts: timestamp (nullable = true)
 |-- record_active_flag: short (nullable = false)
 |-- record_upd_ts: timestamp (nullable = false)
 |-- record_insert_ts: timestamp (nullable = false)



+------------+--------------------------------------------------------------+------------------------------+--------+--------------------+----------+-------------------+-------------------+------------------+-----------------------+-----------------------+
|location_key|address                                                       |street                        |postcode|suburb              |track_hash|record_start_ts    |record_end_ts      |record_active_flag|record_upd_ts          |record_insert_ts       |
+------------+--------------------------------------------------------------+------------------------------+--------+--------------------+----------+-------------------+-------------------+------------------+-----------------------+-----------------------+
|0           |Level 2 85 Rhonda Upper, Jimmyfurt, TAS, 2987                 |Level 2 85 Rhonda Upper       | 2987   | Jimmyfurt          |null      |1970-01-01 00:00:00|2999-12-31 00:00:00|1                 |2023-06-05 08:15:58.0

+----------+-------------+----------+-------------------+-------------------+------------------+-----------------------+-----------------------+
|procedure |procedure_key|track_hash|record_start_ts    |record_end_ts      |record_active_flag|record_upd_ts          |record_insert_ts       |
+----------+-------------+----------+-------------------+-------------------+------------------+-----------------------+-----------------------+
|CT Scan   |0            |null      |1970-01-01 00:00:00|2999-12-31 00:00:00|1                 |2023-06-05 08:16:06.255|2023-06-05 08:16:06.255|
|Blood Test|1            |null      |1970-01-01 00:00:00|2999-12-31 00:00:00|1                 |2023-06-05 08:16:06.255|2023-06-05 08:16:06.255|
|MRI       |2            |null      |1970-01-01 00:00:00|2999-12-31 00:00:00|1                 |2023-06-05 08:16:06.255|2023-06-05 08:16:06.255|
|ECG       |3            |null      |1970-01-01 00:00:00|2999-12-31 00:00:00|1                 |2023-06-05 08:16:06.255|2023-06-05

In [8]:
run3_paths = ['s3://test-lf-wm/etl/optimised/location_dim/full/202306051736/',
       's3://test-lf-wm/etl/optimised/procedure_dim/full/202306051737/']
example_path = run3_paths[-1]
# Crawl
glue.create_crawler(example_path)
# glue.start_crawler(example_path)
glue.wait_for_crawler(example_path)

############## After crawlers finish. #####################
for path in run3_paths:
     
    databrew.create_dataset(path)
    databrew.create_profile_job(path)        
    databrew.start_job_run(path)

############## Wait for job to finish #####################
databrew.wait_for_job(example_path)
time.sleep(10)

# Print links.
for path in run3_paths:
    print(f"You can view the data profile for {path} here:\n\t{databrew.get_profile_link(path)}")
    # databrew.show_data_profile_link(path)




Couldn't create crawler. 618572314333:test-optimised already exists


Crawler not running.
create_table_name: optimisedlocation_dim
create_dataset_name test-optimised-location-dim
create_dataset_name test-optimised-location-dim
create_dataset_name test-optimised-location-dim
create_dataset_name test-optimised-location-dim
create_table_name: optimisedprocedure_dim
create_dataset_name test-optimised-procedure-dim
create_dataset_name test-optimised-procedure-dim
create_dataset_name test-optimised-procedure-dim
create_dataset_name test-optimised-procedure-dim
create_dataset_name test-optimised-procedure-dim
Waiting for job to finish.create_dataset_name test-optimised-procedure-dim
.create_dataset_name test-optimised-procedure-dim
.create_dataset_name test-optimised-procedure-dim
.create_dataset_name test-optimised-procedure-dim
.create_dataset_name test-optimised-procedure-dim
.create_dataset_name test-optimised-procedure-dim
.create_dataset_name test-optimised-procedure-dim
.create_dataset_name test-optimised-procedure-dim
.create_dataset_name test-optimise

In [6]:
glue.create_crawler(['s3://test-lf-wm/etl/optimised/location_dim/full/202306051736/',
       's3://test-lf-wm/etl/optimised/procedure_dim/full/202306051737/'])

Couldn't create crawler. 618572314333:test-optimised already exists


In [15]:

if isinstance(paths, str):
    path_list = [paths]
else:
    path_list = paths
    
s3_targets = [
    {'Path':glue.create_crawler_path(path)}
    for path 
    in path_list
]
print(s3_targets)
example_path = s3_targets[0]['Path']
name = glue.create_crawler_name(example_path)
db_name = glue.create_db_name(example_path)
table_prefix = glue.create_table_prefix(example_path)

print(s3_targets)
    


s3://test-lf-wm/etl/optimised/location_dim/full/202306051736/
['s', '3', ':', '/', '/', 't', 'e', 's', 't', '-', 'l', 'f', '-', 'w', 'm', '/', 'e', 't', 'l', '/', 'o', 'p', 't', 'i', 'm', 'i', 's', 'e', 'd', '/', 'l', 'o', 'c', 'a', 't', 'i', 'o', 'n', '_', 'd', 'i', 'm', '/', 'f', 'u', 'l', 'l', '/', '2', '0', '2', '3', '0', '6', '0', '5', '1', '7', '3', '6', '/']
[{'Path': 's3://test-lf-wm/etl/optimised/location_dim'}, {'Path': 's3://test-lf-wm/etl/optimised/procedure_dim'}]
[{'Path': 's3://test-lf-wm/etl/optimised/location_dim'}, {'Path': 's3://test-lf-wm/etl/optimised/procedure_dim'}]


## Run 4

In [3]:
run4 = []

df, path = stage_policyholder_into_optimised.run(spark,env)
run4.append( (df, path) )
df, path = stage_provider_into_optimised.run(spark,env)
run4.append( (df, path) )

NameError: name 'stage_policyholder_into_optimised' is not defined

In [ ]:
example_path = run4_paths[-1]

# Crawl
glue.create_crawler(example_path)
glue.start_crawler(example_path)
glue.wait_for_crawler(example_path)

############## After crawlers finish. #####################
for path in run4_paths:
     
    databrew.create_dataset(path)
    databrew.create_profile_job(path)        
    databrew.start_job_run(path)

############## Wait for job to finish #####################
databrew.wait_for_job(example_path)
time.sleep(10)

# Print links.
for path in run4_paths:
    print(f"You can view the data profile for {path} here:\n\t{databrew.get_profile_link(path)}")
    # databrew.show_data_profile_link(path)




## Run 5

In [5]:
run5 = []
df, path = stage_claim_into_optimised.run(spark,env)
run5.append( (df, path) )

NameError: name 'stage_claim_into_optimised' is not defined